In [ ]:
%config InlineBackend.figure_format = 'retina'
import findspark
findspark.init()

import pandas as pd
import numpy as np
import os
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext, Row, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import time
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
from slugify import slugify

In [ ]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

authors = sqlContext.read.parquet('./data/authors')
co_networks = sqlContext.read.parquet('./data/coauthors')
collaborations = sqlContext.read.parquet('./data/collaborations')


In [ ]:
saved_columns = [F.first(col).alias(col) for col in authors.columns if col != 'id']

auth_citations = authors.join(collaborations, authors.id == collaborations.author_id, 'inner') \
    .groupby([authors.id]).agg(F.sum(F.col('cited_by')).alias('agg_citations'), *saved_columns)

auth_citations.write.mode('overwrite').parquet(os.path.abspath('./data/authors_agg'))

In [ ]:

authors.describe().show()
auth_citations.describe().show()